In [86]:
from collections import defaultdict
from dataclasses import dataclass
import os
import yaml
import time

import numpy as np
from numpy.typing import NDArray
import scipy.io
import scipy.linalg

from src.linalg import get_scipy_solution

In [87]:
def lu(A: NDArray, permute: bool) -> tuple[NDArray, NDArray, NDArray]:
    n = A.shape[0]
    L = np.eye(n)
    U = np.copy(A)
    P = np.eye(n)

    for column in range(n-1):
        if(permute):
            max_index = np.argmax(abs(U[column:, column])) + column
            if max_index != column:
                U[[column, max_index]] = U[[max_index, column]]
                P[[column, max_index]] = P[[max_index, column]]
                if column != 0:
                    L[[column, max_index], :column] = L[[max_index, column], :column]
            
        a_diag_elem = U[column][column]
        for row in range(column + 1, n):
            coef = U[row][column] / a_diag_elem
            L[row][column] = coef
            print(f'строка = {row} , столбец - {column}, | {U[row]} - {U[column] * coef}')
            U[row] -= (U[column] * coef)
    return L, U, P

In [127]:
def solve(L: NDArray, U: NDArray, P: NDArray, b: NDArray) -> NDArray:
    n = P.shape[0]
    x = np.zeros((n, 1))
    b_permuted = P.dot(b) # меняем местами стобцы b в соответствии с a
    
    # Обозначив вектор y как y = Ux мы находим его как решение уравнения: Ly = Pb = b_permutated
    y = np.zeros((n, 1))
    for i in range(n):
        y[i] = b_permuted[i] -  (L[0:i+1 , 0:i+1] @ y[0:i+1])[i]
    print(y)
    
    # Ux = y
    x = np.zeros((n, 1))
    for i in range(n):
        x[i] = (y[i] - np.sum(U[i, i+1:] * x[i+1:])) / U[i, i]
    return x

In [98]:
b

array([8., 2., 7.])

In [129]:
solve(L,U,P,b)

[[ 8.        ]
 [ 2.2       ]
 [-2.88235294]]


array([[1.6       ],
       [0.64705882],
       [0.77777778]])

In [91]:
A = np.array([[5,1,7],[2,3,2],[3,4,8]] , dtype=np.float64)
b = np.array([8,2,7] , dtype=np.float64)
A

array([[5., 1., 7.],
       [2., 3., 2.],
       [3., 4., 8.]])

In [92]:
L,U,P = lu(A,True)

строка = 1 , столбец - 0, | [2. 3. 2.] - [2.  0.4 2.8]
строка = 2 , столбец - 0, | [3. 4. 8.] - [3.  0.6 4.2]
строка = 2 , столбец - 1, | [ 0.   2.6 -0.8] - [0.         2.6        2.90588235]


In [93]:
L

array([[1.        , 0.        , 0.        ],
       [0.6       , 1.        , 0.        ],
       [0.4       , 0.76470588, 1.        ]])

In [94]:
U

array([[ 5.        ,  1.        ,  7.        ],
       [ 0.        ,  3.4       ,  3.8       ],
       [ 0.        ,  0.        , -3.70588235]])

In [95]:
P

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [96]:
np.linalg.inv(P) @ L @ U

array([[5., 1., 7.],
       [2., 3., 2.],
       [3., 4., 8.]])